In [2]:
import sqlite3
import pandas as pd

In [30]:
conn = sqlite3.connect("../data/ufcstats.db")
conn.execute("ATTACH DATABASE '../data/sherdog.db' AS sherdog")

df = pd.read_sql(
    """
WITH mma_debuts_and_dobs AS (
  SELECT 
    t1.*, 
    t2.DATE_OF_BIRTH, 
    t3.UFCSTATS_FIGHTER_ID 
  FROM 
    (
      SELECT 
        FIGHTER_ID, 
        MIN(DATE) AS SHERDOG_DEBUT_DATE 
      FROM 
        sherdog.SHERDOG_BOUT_HISTORY 
      GROUP BY 
        FIGHTER_ID
    ) AS t1 
    INNER JOIN sherdog.SHERDOG_FIGHTERS AS t2 ON t1.FIGHTER_ID = t2.FIGHTER_ID 
    INNER JOIN sherdog.SHERDOG_FIGHTER_LINKAGE AS t3 ON t1.FIGHTER_ID = t3.SHERDOG_FIGHTER_ID
), 
stacked_sherdog_raw AS (
  SELECT 
    t2.UFCSTATS_FIGHTER_ID, 
    t1.EVENT_ID, 
    t1.FIGHTER_BOUT_ORDINAL, 
    julianday(t1.DATE) - julianday(t2.DATE_OF_BIRTH) AS AGE_DAYS, 
    julianday(t1.DATE) - julianday(t2.SHERDOG_DEBUT_DATE) AS DAYS_SINCE_DEBUT, 
    julianday(t1.DATE) - julianday(
      LAG(t1.DATE, 1) OVER (
        PARTITION BY t1.FIGHTER_ID 
        ORDER BY 
          t1.FIGHTER_BOUT_ORDINAL
      )
    ) AS DAYS_SINCE_LAST_FIGHT, 
    CASE t1.OUTCOME WHEN 'W' THEN 1 ELSE 0 END AS WIN, 
    CASE t1.OUTCOME WHEN 'L' THEN 1 ELSE 0 END AS LOSS, 
    CASE WHEN t1.OUTCOME = 'W' 
    AND t1.OUTCOME_METHOD IN ('KO', 'TKO') THEN 1 ELSE 0 END AS WIN_BY_KO_TKO, 
    CASE WHEN t1.OUTCOME = 'W' 
    AND t1.OUTCOME_METHOD IN (
      'Submission', 'Technical Submission'
    ) THEN 1 ELSE 0 END AS WIN_BY_SUBMISSION, 
    CASE WHEN t1.OUTCOME = 'W' 
    AND t1.OUTCOME_METHOD = 'Decision' THEN 1 ELSE 0 END AS WIN_BY_DECISION, 
    CASE WHEN t1.OUTCOME = 'L' 
    AND t1.OUTCOME_METHOD IN ('KO', 'TKO') THEN 1 ELSE 0 END AS LOSS_BY_KO_TKO, 
    CASE WHEN t1.OUTCOME = 'L' 
    AND t1.OUTCOME_METHOD IN (
      'Submission', 'Technical Submission'
    ) THEN 1 ELSE 0 END AS LOSS_BY_SUBMISSION, 
    CASE WHEN t1.OUTCOME = 'L' 
    AND t1.OUTCOME_METHOD = 'Decision' THEN 1 ELSE 0 END AS LOSS_BY_DECISION, 
    t1.FIGHTER_BOUT_ORDINAL AS TOTAL_FIGHTS, 
    t1.TOTAL_TIME_SECONDS 
  FROM 
    sherdog.SHERDOG_BOUT_HISTORY AS t1 
    LEFT JOIN mma_debuts_and_dobs AS t2 ON t1.FIGHTER_ID = t2.FIGHTER_ID
), 
stacked_sherdog_features AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY UFCSTATS_FIGHTER_ID ORDER BY TOTAL_FIGHTS) AS FIGHTER_BOUT_NUMBER
  FROM (
    SELECT 
      UFCSTATS_FIGHTER_ID, 
      EVENT_ID, 
      AGE_DAYS, 
      AVG(AGE_DAYS) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS AGE_DAYS_AVERAGE, 
      DAYS_SINCE_DEBUT, 
      AVG(DAYS_SINCE_DEBUT) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS DAYS_SINCE_DEBUT_AVERAGE, 
      DAYS_SINCE_LAST_FIGHT, 
      AVG(DAYS_SINCE_LAST_FIGHT) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND CURRENT ROW
      ) AS DAYS_SINCE_LAST_FIGHT_AVERAGE, 
      SUM(WIN) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WINS, 
      AVG(WIN) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WIN_RATE, 
      SUM(LOSS) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSSES, 
      AVG(LOSS) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSS_RATE, 
      SUM(WIN_BY_KO_TKO) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WINS_BY_KO_TKO, 
      AVG(WIN_BY_KO_TKO) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WIN_RATE_BY_KO_TKO, 
      SUM(WIN_BY_SUBMISSION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WINS_BY_SUBMISSION, 
      AVG(WIN_BY_SUBMISSION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WIN_RATE_BY_SUBMISSION, 
      SUM(WIN_BY_DECISION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WINS_BY_DECISION, 
      AVG(WIN_BY_DECISION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS WIN_RATE_BY_DECISION, 
      SUM(LOSS_BY_KO_TKO) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSSES_BY_KO_TKO, 
      AVG(LOSS_BY_KO_TKO) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSS_RATE_BY_KO_TKO, 
      SUM(LOSS_BY_SUBMISSION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSSES_BY_SUBMISSION, 
      AVG(LOSS_BY_SUBMISSION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSS_RATE_BY_SUBMISSION, 
      SUM(LOSS_BY_DECISION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSSES_BY_DECISION, 
      AVG(LOSS_BY_DECISION) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS LOSS_RATE_BY_DECISION, 
      TOTAL_FIGHTS, 
      SUM(TOTAL_TIME_SECONDS) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS TIME_SECONDS_FOUGHT, 
      AVG(TOTAL_TIME_SECONDS) OVER (
        PARTITION BY UFCSTATS_FIGHTER_ID 
        ORDER BY 
          FIGHTER_BOUT_ORDINAL ROWS BETWEEN UNBOUNDED PRECEDING 
          AND 1 PRECEDING
      ) AS TIME_SECONDS_FOUGHT_AVERAGE 
    FROM 
      stacked_sherdog_raw
  )
  WHERE
    EVENT_ID IN (
      SELECT
        EVENT_ID
      FROM
        sherdog.SHERDOG_BOUTS
    )
)
select * from stacked_sherdog_features
    """,
    conn,
)

conn.close()

In [31]:
df

,UFCSTATS_FIGHTER_ID,EVENT_ID,AGE_DAYS,AGE_DAYS_AVERAGE,DAYS_SINCE_DEBUT,DAYS_SINCE_DEBUT_AVERAGE,DAYS_SINCE_LAST_FIGHT,DAYS_SINCE_LAST_FIGHT_AVERAGE,WINS,WIN_RATE,...,LOSSES_BY_KO_TKO,LOSS_RATE_BY_KO_TKO,LOSSES_BY_SUBMISSION,LOSS_RATE_BY_SUBMISSION,LOSSES_BY_DECISION,LOSS_RATE_BY_DECISION,TOTAL_FIGHTS,TIME_SECONDS_FOUGHT,TIME_SECONDS_FOUGHT_AVERAGE,FIGHTER_BOUT_NUMBER
0,002ca196477ce572,76575,9094.0,7237.375000,2793.0,936.375000,113.0,349.125000,8.0,1.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,8,4382.0,547.750000,1
1,002ca196477ce572,83479,9318.0,7443.666667,3017.0,1142.666667,224.0,335.222222,8.0,0.888889,...,0.0,0.000000,0.0,0.000000,1.0,0.111111,9,5282.0,586.888889,2
2,003d82fa384ca1d0,83479,11119.0,9868.600000,2304.0,1053.600000,214.0,230.400000,8.0,0.800000,...,0.0,0.000000,2.0,0.200000,0.0,0.000000,10,6654.0,665.400000,1
3,003d82fa384ca1d0,88003,11490.0,9982.272727,2675.0,1167.272727,371.0,243.181818,8.0,0.727273,...,1.0,0.090909,2.0,0.181818,0.0,0.000000,11,6739.0,612.636364,2
4,0052de90691d4a93,56257,11084.0,10059.714286,2442.0,1417.714286,91.0,348.857143,6.0,0.857143,...,0.0,0.000000,0.0,0.000000,1.0,0.142857,7,2333.0,388.833333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,ffe9703408fb5964,89999,10000.0,9454.375000,973.0,427.375000,15.0,121.625000,8.0,1.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,8,2090.0,261.250000,1
15150,ffe9703408fb5964,90843,10119.0,9515.000000,1092.0,488.000000,119.0,121.333333,8.0,0.888889,...,0.0,0.000000,0.0,0.000000,1.0,0.111111,9,2990.0,332.222222,2
15151,ffe9703408fb5964,92738,10259.0,9575.400000,1232.0,548.400000,140.0,123.200000,9.0,0.900000,...,0.0,0.000000,0.0,0.000000,1.0,0.100000,10,3254.0,325.400000,3
15152,ffe9703408fb5964,93192,10294.0,9637.545455,1267.0,610.545455,35.0,115.181818,10.0,0.909091,...,0.0,0.000000,0.0,0.000000,1.0,0.090909,11,3747.0,340.636364,4
